<a href="https://colab.research.google.com/github/tazar09/un_inputs/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import pandas as pd
pd.options.display.max_rows = 601

import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re

#pip install PyPDF2
#pip install langdetect

from PyPDF2 import PdfReader
from textblob import TextBlob

import time
from langdetect import detect

In [6]:
url = 'https://financing.desa.un.org/un-tax-convention/inputs'

In [27]:
def scrape_link(url):
    pdf_data = []
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.content, 'html.parser')

      for link in soup.find_all('a', href=lambda href: href and href.endswith('.pdf')):
       text = link.text.strip()  # Extract link text (attribute)
       url = link.get('href')  # Extract PDF URL from 'href' attribute
       pdf_data.append({'text': text, 'url': url})

    # Create a Pandas DataFrame
      df = pd.DataFrame(pdf_data)
      return df

    else:
      print(f"Error: Website request failed with status code {response.status_code}")
      return none

In [105]:
def scrape_and_analyze_pdfs(url):
  # Data structure to store results
  data = []
  delay = 3

  # Scrape PDF links and attributes
  pdf_data = scrape_link(url = url)

  # Download and analyze each PDF
  if pdf_data is not None:
    for index, row in pdf_data.iterrows():
      pdf_url = row['url']
      text = row['text']

      # Download the PDF (replace with error handling)
      response = requests.get(pdf_url, stream=True)
      if response.status_code == 200:
        with open(f"pdf_{index+1}.pdf", 'wb') as f:
          for chunk in response.iter_content(1024):
            f.write(chunk)

        time.sleep(delay)
        # Extract text using PyPDF2 (replace with more advanced NLP techniques)
        reader = PdfReader(f"pdf_{index+1}.pdf")
        extracted_text = ""
        for page in reader.pages:
          extracted_text += page.extract_text()

        extracted_text = extracted_text.strip()
        extracted_text = re.sub(r'\s+', ' ', extracted_text)

        data.append({'text': text, 'url': pdf_url, 'extracted_text': extracted_text})

      # Remove downloaded PDF (optional)
      # os.remove(f"pdf_{index+1}.pdf")

  # Create a Pandas DataFrame
  df = pd.DataFrame(data)
  return df

In [106]:
# Call the function with your target URL (replace with your actual URL)
target_url = url
pdf_data = scrape_and_analyze_pdfs(target_url)

In [107]:
# pdf_data['extracted_text'] = pdf_data['extracted_text'].apply(lambda x: re.sub(r'\s+', ' ',x))
# pdf_data['extracted_text'] = pdf_data['extracted_text'].str.strip()

def extract_country(text):
  match = re.search(r"(?<=2024-03\/)([^_]+)(?=_Input)", text)
  return match.group(1) if match else None  # Handle cases where no match is found

pdf_data['country'] = pdf_data['url'].apply(extract_country)

pdf_data['country'] = pdf_data['country'].str.replace("%", ' ')
pdf_data['country'] = pdf_data['country'].str.replace("20", '')
pdf_data['country'] = pdf_data['country'].str.strip()




In [109]:
pdf_data = pdf_data[['text', 'country', 'url', 'extracted_text']]
pdf_data.head()

In [ ]:
pdf = pdf_data

In [149]:
pdf.loc[pdf['text'] == '[ENG]', 'text'] = pdf['country'].where(pdf['text'] == '[ENG]')
pdf['sentiment'] = pdf['extracted_text'].apply(lambda x:TextBlob(x).sentiment.polarity)

In [164]:
def detect_language(text):
  try:
    return detect(text)
  except:
    return None  # Handle potential errors during language detection

pdf['language'] = pdf['extracted_text'].apply(detect_language)

In [196]:
pdf = pdf[['text', 'country', 'url', 'extracted_text', 'language', 'sentiment']]

In [209]:
pdf.head()

,text,country,url,extracted_text,language,sentiment
0,African Group,African Group,https://financing.desa.un.org/sites/default/fi...,,None,0.000000
1,Austria,Austria,https://financing.desa.un.org/sites/default/fi...,Invitation to provide inputs to the work of th...,en,0.127652
2,Bahamas,Bahamas,https://financing.desa.un.org/sites/default/fi...,The Commonwealth of The Bahamas Input on UN Fr...,en,0.116546
3,Belarus,Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,en,0.087142
4,[RUS],Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,ru,0.077273


In [208]:
pdf.to_excel('inputs.xlsx')

In [210]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')

In [252]:
def analyze_sentiment(text):

  # Create VADER sentiment analyzer
  analyzer = SentimentIntensityAnalyzer()

  # Analyze sentiment
  sentiment = analyzer.polarity_scores(text)

  return sentiment

In [253]:
def sentiment_analysis_df(data):

  # Add new columns for sentiment scores
  data['sentiment'] = data['extracted_text'].apply(analyze_sentiment)
  data['compound'] = data['sentiment'].apply(lambda x: x['compound'])
  data['pos'] = data['sentiment'].apply(lambda x: x['pos'])
  data['neg'] = data['sentiment'].apply(lambda x: x['neg'])
  data['neu'] = data['sentiment'].apply(lambda x: x['neu'])  # Extract neutral score

  # Return the DataFrame with sentiment scores
  return data

In [255]:
pdf_sent = sentiment_analysis_df(pdf.copy())  # Avoid modifying original DataFrame
pdf_sent.head()

,text,country,url,extracted_text,language,sentiment,compound,pos,neg,neu
0,African Group,African Group,https://financing.desa.un.org/sites/default/fi...,,None,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",0.0000,0.000,0.000,0.000
1,Austria,Austria,https://financing.desa.un.org/sites/default/fi...,Invitation to provide inputs to the work of th...,en,"{'neg': 0.0, 'neu': 0.875, 'pos': 0.125, 'comp...",0.9957,0.125,0.000,0.875
2,Bahamas,Bahamas,https://financing.desa.un.org/sites/default/fi...,The Commonwealth of The Bahamas Input on UN Fr...,en,"{'neg': 0.037, 'neu': 0.819, 'pos': 0.144, 'co...",0.9996,0.144,0.037,0.819
3,Belarus,Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,en,"{'neg': 0.012, 'neu': 0.903, 'pos': 0.085, 'co...",0.9969,0.085,0.012,0.903
4,[RUS],Belarus,https://financing.desa.un.org/sites/default/fi...,№ й'.-ЛрШгPERMANENT MISSION OF THE REPUBLIC OF...,ru,"{'neg': 0.0, 'neu': 0.967, 'pos': 0.033, 'comp...",0.9810,0.033,0.000,0.967


In [258]:
pdf_sent.sort_values(by = 'neg', ascending = False)

,text,country,url,extracted_text,language,sentiment,compound,pos,neg,neu
82,International Bureau for Fiscal Documentation ...,International Bureau for Fiscal Documentation ...,https://financing.desa.un.org/sites/default/fi...,"1 IBFD INPUT (Pasquale Pistone, Ivan Lazarov, ...",en,"{'neg': 0.094, 'neu': 0.79, 'pos': 0.116, 'com...",0.9893,0.116,0.094,0.790
86,"Michel, Bob",Michel 2C Bob,https://financing.desa.un.org/sites/default/fi...,Dispute resolution under a Framework Conventio...,en,"{'neg': 0.078, 'neu': 0.807, 'pos': 0.114, 'co...",0.9976,0.114,0.078,0.807
62,Act Alliance,Act Alliance,https://financing.desa.un.org/sites/default/fi...,Subject: submission in response to the call fo...,en,"{'neg': 0.07, 'neu': 0.783, 'pos': 0.147, 'com...",0.9959,0.147,0.070,0.783
53,UN Enivronmental Programme,UN Environment Programme,https://financing.desa.un.org/sites/default/fi...,Proposed inputs from the Industry and Economy ...,en,"{'neg': 0.069, 'neu': 0.771, 'pos': 0.16, 'com...",0.9969,0.160,0.069,0.771
105,KPMG,KPMG,https://financing.desa.un.org/sites/default/fi...,From: KPMG International By email Dear Chair o...,en,"{'neg': 0.058, 'neu': 0.708, 'pos': 0.234, 'co...",0.9990,0.234,0.058,0.708
52,Independent Expert on effects of foreign debt ...,Independent Expert on effects of foreign debt ...,https://financing.desa.un.org/sites/default/fi...,Ms. Attiya Waris Independent Expert on effect ...,en,"{'neg': 0.054, 'neu': 0.843, 'pos': 0.103, 'co...",0.9876,0.103,0.054,0.843
63,African Parliamentary Network on Illicit Finan...,African Parliamentary Network on Illicit Finan...,https://financing.desa.un.org/sites/default/fi...,"1Max Fuss From: AHC-TAX Sent: Monday, 11 March...",en,"{'neg': 0.054, 'neu': 0.876, 'pos': 0.07, 'com...",0.7506,0.070,0.054,0.876
70,Civil Society Bahamas (CSB),Civil Society Bahamas 28CSB 29,https://financing.desa.un.org/sites/default/fi...,"P.O. Box N -1353 Nassau, The Bahamas CELL: 242...",en,"{'neg': 0.052, 'neu': 0.842, 'pos': 0.106, 'co...",0.9760,0.106,0.052,0.842
67,Center for Economic and Social Rights (CESR),Center for Economic and Social Rights 28CESR 29,https://financing.desa.un.org/sites/default/fi...,1 Ad Hoc Committee to Draft Terms of Reference...,en,"{'neg': 0.05, 'neu': 0.845, 'pos': 0.106, 'com...",0.9988,0.106,0.050,0.845
68,Center for the Study of Law 2C Justice 2C and ...,Center for the Study of Law 2C Justice 2C and ...,https://financing.desa.un.org/sites/default/fi...,"Bogotá, Colombia. March 14, 2024 Mr. Ramy Yous...",en,"{'neg': 0.049, 'neu': 0.809, 'pos': 0.142, 'co...",0.9995,0.142,0.049,0.809
